# Modélisation BERT pour classification des images par leur OCR

In [ ]:
# Pour l'OCR avec EasyOCR
!pip install easyocr

# Pour les opérations avec PyTorch (déjà inclus dans Google Colab, mais si vous êtes sur un environnement local)
!pip install torch torchvision torchaudio

# Pour utiliser Hugging Face Transformers (pour le modèle BERT et le tokenizer)
!pip install transformers

# Pour manipuler des datasets avec Hugging Face (facultatif mais utile pour gérer les datasets)
!pip install datasets

# Pour travailler avec des DataFrames (par exemple pour `df_ocr`)
!pip install pandas

# Pour la division des jeux de données (train/test split)
!pip install scikit-learn

In [ ]:
import pandas as pd

#-------------------------------------------------------------------------------
#paramétrage de lancement
#-------------------------------------------------------------------------------
results_dir = '/content/drive/MyDrive/formation Datascientest/RVL-CDIP/'
csv_name = 'processed_ocr_results_reducedtrain.csv'
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
df_ocr = pd.read_csv(results_dir + csv_name)


# Prétraitement du texte pour le pipeline NLP
Nous allons utiliser le tokenizer BERT pour préparer les données textuelles contenues dans la colonne texte_de_ocr de votre DataFrame. Voici comment effectuer le prétraitement des données.

Charger le tokenizer BERT.
Appliquer le tokenizer sur chaque texte de la DataFrame.
Préparer les données pour être compatibles avec un modèle BERT.

In [ ]:
from transformers import BertTokenizer
import torch

# Charger le tokenizer BERT pré-entraîné
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Fonction pour tokeniser le texte
def preprocess_texts(texts):
    # Tokeniser tous les textes dans un seul batch
    tokens = tokenizer(
        texts.tolist(),  # Convertir la colonne en liste
        padding='max_length',  # Remplir les séquences pour qu'elles aient toutes la même longueur
        truncation=True,       # Troncation pour respecter la longueur maximale
        max_length=512,        # Longueur maximale compatible avec BERT
        return_tensors='pt'    # Retourner les tenseurs PyTorch
    )
    return tokens

# Appliquer le prétraitement sur le texte de la DataFrame
tokens = preprocess_texts(df_ocr['texte_de_ocr'])

# Classification des documents avec BERT
Maintenant que nous avons prétraité les textes, nous allons construire un modèle de classification basé sur BERT. Le modèle va apprendre à prédire les labels des documents à partir des textes extraits.

Charger un modèle BERT pré-entraîné pour la classification.
Préparer les données pour l'entraînement (input_ids, attention_masks, labels).
Entraîner et évaluer le modèle.
5.1. Préparation des données pour l'entraînement
Nous devons associer les textes tokenisés à leurs labels et préparer les tenseurs pour l'entraînement.

In [ ]:
from torch.utils.data import Dataset, DataLoader

# Préparer un dataset personnalisé pour PyTorch
class RVLCDIPDataset(Dataset):
    def __init__(self, tokens, labels):
        self.input_ids = tokens['input_ids']
        self.attention_mask = tokens['attention_mask']
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# Créer le dataset à partir des tokens et des labels dans la DataFrame
dataset = RVLCDIPDataset(tokens, df_ocr['label_de_image'].tolist())

# Diviser en train et test (80% train, 20% test)
from sklearn.model_selection import train_test_split
train_indices, test_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=42)

# Créer des DataLoader pour l'entraînement et l'évaluation
train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


# Entraînement du modèle BERT pour la classification
Nous allons maintenant charger un modèle BERT pour la classification de séquences et l'entraîner sur le dataset tokenisé.

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Charger le modèle BERT pré-entraîné pour la classification avec 16 labels
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=16)

# Préparer les arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',          # Répertoire de sortie
    evaluation_strategy="epoch",     # Évaluer à chaque époque
    per_device_train_batch_size=16,  # Taille de batch pour l'entraînement
    per_device_eval_batch_size=16,   # Taille de batch pour l'évaluation
    num_train_epochs=3,              # Nombre d'époques d'entraînement
    weight_decay=0.01,               # Taux de régularisation
    logging_dir='./logs',            # Répertoire des logs
)

# Créer un objet Trainer pour gérer l'entraînement
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Entraîner le modèle
trainer.train()

# Évaluer le modèle
trainer.evaluate()

# Effectuer l'évaluation
results = trainer.evaluate()
print(f"Evaluation results: {results}")

# Sauvegarder les résultats dans un fichier texte
with open("evaluation_results.txt", "w") as f:
    for key, value in results.items():
        f.write(f"{key}: {value}\n")

print("Les résultats d'évaluation ont été sauvegardés dans 'evaluation_results.txt'.")

import json
# Sauvegarder les résultats dans un fichier JSON
with open("evaluation_results.json", "w") as f:
    json.dump(results, f, indent=4)

print("Les résultats d'évaluation ont été sauvegardés dans 'evaluation_results.json'.")


# sauvegarde du modèle

In [ ]:
import os
import shutil

# Spécifier le répertoire où enregistrer le modèle
output_dir = 'content/saved_model/'

# Vérifier si le répertoire existe, sinon le créer
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Répertoire {output_dir} créé avec succès.")
else:
    print(f"Le répertoire {output_dir} existe déjà.")

# Sauvegarder le modèle entraîné
model.save_pretrained(output_dir)

# Sauvegarder le tokenizer utilisé pour la tokenisation
tokenizer.save_pretrained(output_dir)

print(f"Le modèle et le tokenizer ont été sauvegardés dans le répertoire {output_dir}")

drive_model_dir = '/content/drive/MyDrive/formation Datascientest/jul24_bds_extraction/ETAPE 2/saved_model/'
# Copier le répertoire local vers Google Drive
shutil.copytree(output_dir, drive_model_dir, dirs_exist_ok=True)

print(f"Le répertoire {output_dir} a été copié sur Google Drive à {drive_model_dir}.")

# Rechargement du modele

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

# Recharger le modèle à partir du répertoire
model = BertForSequenceClassification.from_pretrained(drive_model_dir)

# Recharger le tokenizer
tokenizer = BertTokenizer.from_pretrained(drive_model_dir)

print("Le modèle et le tokenizer ont été rechargés avec succès.")